# Basics

Import libraries and functions.

</p>Search for archives, and create the 1st data frame, as well as an empty list for the headers (h) and the list for comun columns (CC)

<p>Using the <code>glob</code> library allows us to create a list with only the csv that are ended in <em>(Normalized).csv</em> , which will be the most useful for a statistical analysis. <br>
Moreover, for a future simplification, we also create a list with the relevant information of each of the csv <em>('Area','Year','Element','Unit','Value')</em> </p>

In [19]:
import pandas as pd
import numpy as np
import glob
from functools import reduce


In [20]:
folder_path = 'C:/Users/amarchve/Desktop/Data'
file_list = glob.glob(folder_path + "/**(Normalized).csv")
CC=['Area','Year','Element','Unit','Value']

# Data integration

As at the beginning we do not have enough computer power to process the whole database, we are going to develop a test run to be sure that the ideas are escalable.


In this first test run for the loop, we are going to load and process the data into concated data frames, where later on, there is an application of the <code>pivot_table</code> function which adjusts all the variables, previouly called 'Elements' & 'Units' into the headers of the columns, and the values the values of the table.

In [9]:
main_dataframe = pd.DataFrame(pd.read_csv(file_list[0], sep=',', encoding='latin-1'),columns=CC)
for i in range(1,5):
    df = pd.DataFrame(pd.read_csv(file_list[i],sep=',' , encoding='latin-1',low_memory=False), columns=CC)
    main_dataframe = pd.concat([main_dataframe, df])

main_dataframeC=main_dataframe.pivot_table(index=['Area','Year'], columns= ['Element','Unit'], values='Value')
main_dataframeC

Element          Domestic supply quantity Export Quantity      Feed  \
Unit                               tonnes          tonnes    tonnes   
Area        Year                                                      
Afghanistan 1961             49168.461538        3819.250   99751.6   
            1962             52625.153846        4445.875  100276.0   
            1963             53554.846154        4881.125   92627.8   
            1964             53641.615385        5224.000  103823.8   
            1965             55104.153846        4015.875  106227.2   
...                                   ...             ...       ...   
Zimbabwe    2016                      NaN             NaN       NaN   
            2017                      NaN             NaN       NaN   
            2018                      NaN             NaN       NaN   
            2019                      NaN             NaN       NaN   
            2020                      NaN             NaN       NaN   

Element          Food supply quantity (tonnes) Import Quantity  Losses  \
Unit                                    tonnes          tonnes  tonnes   
Area        Year                                                         
Afghanistan 1961                        4900.0       22.666667  1100.0   
            1962                        4965.0       10.666667  1138.0   
            1963                        4950.0       12.166667  1136.0   
            1964                        4965.0       10.666667  1138.0   
            1965                        5100.0        7.500000  1184.0   
...                                        ...             ...     ...   
Zimbabwe    2016                           NaN             NaN     NaN   
            2017                           NaN             NaN     NaN   
            2018                           NaN             NaN     NaN   
            2019                           NaN             NaN     NaN   
            2020                           NaN             NaN     NaN   

Element          Other uses (non-food) Per 100,000 farmers Processing  \
Unit                            tonnes                 FTE     tonnes   
Area        Year                                                        
Afghanistan 1961                7406.4                 NaN    41987.5   
            1962                9284.8                 NaN    57303.0   
            1963               10943.0                 NaN    78337.0   
            1964                8170.2                 NaN    59243.0   
            1965                9368.4                 NaN    60740.5   
...                                ...                 ...        ...   
Zimbabwe    2016                   NaN                 6.5        NaN   
            2017                   NaN                 NaN        NaN   
            2018                   NaN                 NaN        NaN   
            2019                   NaN                 NaN        NaN   
            2020                   NaN                 NaN        NaN   

Element          Production Researchers, total    Seed  \
Unit                 tonnes                FTE  tonnes   
Area        Year                                         
Afghanistan 1961    66960.8                NaN  4262.5   
            1962    71963.0                NaN  5324.5   
            1963    73518.9                NaN  5324.5   
            1964    73906.9                NaN  3908.5   
            1965    74843.6                NaN  2755.5   
...                     ...                ...     ...   
Zimbabwe    2016        NaN              242.0     NaN   
            2017        NaN                NaN     NaN   
            2018        NaN                NaN     NaN   
            2019        NaN                NaN     NaN   
            2020        NaN                NaN     NaN   

Element          Share of Value Added (Agriculture, Forestry and Fishing)  \
Unit                                                   

<p>In the following cell, we are concatanating all the files from the <code>file_list</code>, which will have the same shape thanks to creation of the dataframes with the restriction of the columns. <br>
Moreover, this concat function will allow for a single data frame which has all the files one on top of another. Therefore the final result form this loop will be <code>main_dataframe</code> which will be our Normalized Source Data Model.</p>

In [ ]:
main_dataframe = pd.DataFrame(pd.read_csv(file_list[0], sep=',', encoding='latin-1'),columns=CC)
for i in range(1,len(file_list)):
    df = pd.DataFrame(pd.read_csv(file_list[i],sep=',' , encoding='latin-1',low_memory=False), columns=CC)
    main_dataframe = pd.concat([main_dataframe, df])

<p>Lastly, to convert the Normalized Source Data Model into the Normalized Integrated Data Model, we are going to use the <code>pivot_table</code> function which allows to <br>
adjusts all the variables, previouly called <em>'Elements' & 'Units'</em> into the headers of the columns, and the <em>'Value'</em> column will be the values of the table.

In [ ]:
main_dataframeC=main_dataframe.pivot_table(index=['Area','Year'], columns= ['Element','Unit'], values='Value')
main_dataframeC

# Quality assurance

In the following cell, we are going to make sure that none of our interesting variables from <em>'Elements'</em> have been left out, thus checking if the extraction & integtration has been completed.

In [11]:
if len(main_dataframe["Element"].value_counts())==main_dataframeC.shape[1]:
    print('Data extraction & integration is COMPLETED and CORRECT')
else:
    print('Data extraction & integration is UNCOMPLETED')

Data extraction & integration is COMPLETED and CORRECT
